# HAR CNN training 

In [292]:
# Imports
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

In [293]:
%run utils/utilities.py

In [294]:
class_label_vn = [u"Cổ tay",u"Cổ chân",u"Bả vai",u"Xoay người",u"Xoay đầu gối",u"Đi bộ",u"Chạy",u"Đá bóng",u"Đạp",u"Đánh răng",u"Rửa tay",u"Lau bàn",u"Quét nhà",u"Nạo",u"Thái",u"Trộn",u"Lên cầu thang",u"Xuống cầu thang"]

## Prepare data

In [295]:
rootDatasetDir = "./datasets/PTIT/normalized"
X_train, labels_train, list_ch_train = read_data(data_path="./datasets/PTIT/normalized", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path=rootDatasetDir, split="test") # test
assert list_ch_train == list_ch_test, "Mistmatch in channels!"

x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr
x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr


In [296]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [297]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [298]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [299]:
# Imports
import tensorflow as tf

### Hyperparameters

In [300]:
batch_size = 600       # Batch size
seq_len = WINDOWN_SIZE          # Number of steps or window size
learning_rate = 0.0001
epochs = 1000

n_classes = NUM_CLASS
n_channels = NUM_CHANNEL
print "n_channels %d" % n_channels

n_channels 9


### Construct the graph
Placeholders

In [301]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

In [302]:
with graph.as_default():
    # (batch, 128, 9) --> (batch, 64, 18)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Now, flatten and pass to the classifier

In [303]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_4, (-1, 8*144))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [ ]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

('Epoch: 0/1000', 'Iteration: 5', 'Train loss: 2.931217', 'Train acc: 0.041667')
('Epoch: 1/1000', 'Iteration: 10', 'Train loss: 2.894500', 'Train acc: 0.063333')
('Epoch: 1/1000', 'Iteration: 10', 'Validation loss: 2.842135', 'Validation acc: 0.073333')
('Epoch: 2/1000', 'Iteration: 15', 'Train loss: 2.859332', 'Train acc: 0.071667')
('Epoch: 3/1000', 'Iteration: 20', 'Train loss: 2.820496', 'Train acc: 0.070000')
('Epoch: 3/1000', 'Iteration: 20', 'Validation loss: 2.785660', 'Validation acc: 0.081667')
('Epoch: 4/1000', 'Iteration: 25', 'Train loss: 2.805601', 'Train acc: 0.091667')
('Epoch: 5/1000', 'Iteration: 30', 'Train loss: 2.803388', 'Train acc: 0.081667')
('Epoch: 5/1000', 'Iteration: 30', 'Validation loss: 2.743469', 'Validation acc: 0.081667')
('Epoch: 6/1000', 'Iteration: 35', 'Train loss: 2.776217', 'Train acc: 0.076667')
('Epoch: 7/1000', 'Iteration: 40', 'Train loss: 2.757503', 'Train acc: 0.095000')
('Epoch: 7/1000', 'Iteration: 40', 'Validation loss: 2.707323', 'Vali

('Epoch: 64/1000', 'Iteration: 325', 'Train loss: 1.456309', 'Train acc: 0.510000')
('Epoch: 65/1000', 'Iteration: 330', 'Train loss: 1.410827', 'Train acc: 0.526667')
('Epoch: 65/1000', 'Iteration: 330', 'Validation loss: 1.339897', 'Validation acc: 0.603333')
('Epoch: 66/1000', 'Iteration: 335', 'Train loss: 1.473915', 'Train acc: 0.491667')
('Epoch: 67/1000', 'Iteration: 340', 'Train loss: 1.426582', 'Train acc: 0.508333')
('Epoch: 67/1000', 'Iteration: 340', 'Validation loss: 1.310915', 'Validation acc: 0.610000')
('Epoch: 68/1000', 'Iteration: 345', 'Train loss: 1.399155', 'Train acc: 0.541667')
('Epoch: 69/1000', 'Iteration: 350', 'Train loss: 1.371071', 'Train acc: 0.516667')
('Epoch: 69/1000', 'Iteration: 350', 'Validation loss: 1.280630', 'Validation acc: 0.613333')
('Epoch: 70/1000', 'Iteration: 355', 'Train loss: 1.330434', 'Train acc: 0.575000')
('Epoch: 71/1000', 'Iteration: 360', 'Train loss: 1.311707', 'Train acc: 0.566667')
('Epoch: 71/1000', 'Iteration: 360', 'Validati

('Epoch: 127/1000', 'Iteration: 640', 'Train loss: 0.806685', 'Train acc: 0.741667')
('Epoch: 127/1000', 'Iteration: 640', 'Validation loss: 0.817582', 'Validation acc: 0.756667')
('Epoch: 128/1000', 'Iteration: 645', 'Train loss: 0.819939', 'Train acc: 0.731667')
('Epoch: 129/1000', 'Iteration: 650', 'Train loss: 0.816517', 'Train acc: 0.741667')
('Epoch: 129/1000', 'Iteration: 650', 'Validation loss: 0.813389', 'Validation acc: 0.751667')
('Epoch: 130/1000', 'Iteration: 655', 'Train loss: 0.816526', 'Train acc: 0.725000')
('Epoch: 131/1000', 'Iteration: 660', 'Train loss: 0.843561', 'Train acc: 0.718333')
('Epoch: 131/1000', 'Iteration: 660', 'Validation loss: 0.801367', 'Validation acc: 0.763333')
('Epoch: 132/1000', 'Iteration: 665', 'Train loss: 0.803931', 'Train acc: 0.745000')
('Epoch: 133/1000', 'Iteration: 670', 'Train loss: 0.795558', 'Train acc: 0.728333')
('Epoch: 133/1000', 'Iteration: 670', 'Validation loss: 0.792569', 'Validation acc: 0.770000')
('Epoch: 134/1000', 'Iter

('Epoch: 189/1000', 'Iteration: 950', 'Train loss: 0.561135', 'Train acc: 0.815000')
('Epoch: 189/1000', 'Iteration: 950', 'Validation loss: 0.632802', 'Validation acc: 0.813333')
('Epoch: 190/1000', 'Iteration: 955', 'Train loss: 0.592072', 'Train acc: 0.800000')
('Epoch: 191/1000', 'Iteration: 960', 'Train loss: 0.587844', 'Train acc: 0.803333')
('Epoch: 191/1000', 'Iteration: 960', 'Validation loss: 0.628549', 'Validation acc: 0.816667')
('Epoch: 192/1000', 'Iteration: 965', 'Train loss: 0.563876', 'Train acc: 0.823333')
('Epoch: 193/1000', 'Iteration: 970', 'Train loss: 0.593133', 'Train acc: 0.808333')
('Epoch: 193/1000', 'Iteration: 970', 'Validation loss: 0.627104', 'Validation acc: 0.811667')
('Epoch: 194/1000', 'Iteration: 975', 'Train loss: 0.582264', 'Train acc: 0.805000')
('Epoch: 195/1000', 'Iteration: 980', 'Train loss: 0.570583', 'Train acc: 0.810000')
('Epoch: 195/1000', 'Iteration: 980', 'Validation loss: 0.622402', 'Validation acc: 0.811667')
('Epoch: 196/1000', 'Iter

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []
with graph.as_default():
    prediction=tf.argmax(logits,1)
with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    y_pred = []
    y_true = []
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        batch_acc, batch_y_pred = sess.run([accuracy, prediction], feed_dict=feed)
        y_pred.extend(batch_y_pred)
        y_true.extend(np.where(r==1)[0][0] for r in y_t )
        
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
#     print y_true
#     print y_pred
    sk_class_labels = [i for i in range(NUM_CLASS)]
    print precision_recall_fscore_support(y_true, y_pred, average=None, labels=sk_class_labels)
    print 'Accuracy:', accuracy_score(y_true, y_pred)
    print 'F1 score:', f1_score(y_true, y_pred, average='micro')
    print 'Recall:', recall_score(y_true, y_pred, average='micro')
    print 'Precision:', precision_score(y_true, y_pred, average='micro')
    print '\n clasification report:\n', classification_report(y_true,y_pred)
    print '\n confussion matrix:\n',confusion_matrix(y_true, y_pred)